In [6]:
import numpy as np
import pandas as pd
from datetime import datetime

from scipy.stats import norm

In [7]:
df_nb = pd.read_csv('NueBevABTest.csv')
df_nb['Customer Begin'] = pd.to_datetime(df_nb['Customer Begin'], errors='coerce')
df_nb['Customer End'] = pd.to_datetime(df_nb['Customer End'], errors='coerce')

In [8]:
df_nb.head(10)

,Client number,Region,Customer Begin,Customer End,Churned,Margin Group,Account Size
0,6025,Los Angeles,2016-06-24,2017-01-27,False,High,Medium
1,7586,Los Angeles,2016-03-10,2017-01-24,False,High,Large
2,9740,Los Angeles,2016-02-26,2016-12-09,True,High,Large
3,1112,Toronto,2016-07-05,2016-10-28,True,Low,Small
4,1223,Toronto,2015-04-27,2017-01-31,False,Low,Medium
5,1249,Toronto,2015-08-26,2017-01-25,False,Low,Medium
6,1318,Toronto,2016-06-01,2016-09-21,True,High,Medium
7,1667,Toronto,2015-09-18,2017-01-20,False,High,Large
8,2090,Toronto,2016-05-12,2016-12-22,True,Low,Medium
9,2267,Toronto,2015-10-19,2016-12-19,True,Low,Large


# Question 1

In [9]:
def check_active(begin, end, check_date):
    return (begin < check_date) and (check_date <= end)

In [10]:
for check_date in [datetime(2016, 9, 9), datetime(2016, 10, 1), datetime(2016, 11, 1),
              datetime(2016, 12, 1), datetime(2017, 1, 1), datetime(2017, 1, 27)]:
    df_nb['active_prior_to_%s' % check_date.strftime("%m%d%y")] = \
        df_nb.apply(lambda line: check_active(line['Customer Begin'], line['Customer End'], check_date), axis=1)


In [12]:
len(df_nb['Client number'].unique())

254

In [11]:
df_nb.sum()

Client number             1426568.0
Churned                        46.0
active_prior_to_090916        253.0
active_prior_to_100116        246.0
active_prior_to_110116        227.0
active_prior_to_120116        217.0
active_prior_to_010117        208.0
active_prior_to_012717        145.0
dtype: float64

In [164]:
print "September churn rate:", 1-df_nb.sum()['active_prior_to_100116'] / df_nb.sum()['active_prior_to_090916']
print "October churn rate:", 1-df_nb.sum()['active_prior_to_110116'] / df_nb.sum()['active_prior_to_100116']
print "November churn rate:", 1-df_nb.sum()['active_prior_to_120116'] / df_nb.sum()['active_prior_to_110116']
print "December churn rate:", 1-df_nb.sum()['active_prior_to_010117'] / df_nb.sum()['active_prior_to_120116']
print "January churn rate:", 1-df_nb.sum()['active_prior_to_012717'] / df_nb.sum()['active_prior_to_010117']

print "Overall monthly churn rate:", 1-sum(df_nb.sum()[3:8]) / sum(df_nb.sum()[2:7])

September churn rate: 0.0276679841897
October churn rate: 0.0772357723577
November churn rate: 0.0440528634361
December churn rate: 0.0414746543779
January churn rate: 0.302884615385
Overall monthly churn rate: 0.0938314509123


In [165]:
df_active_both = df_nb.sum()

In [170]:
churn_rates = [1-df_active_both[i+1]/df_active_both[i] for i in range(2,7)]

In [187]:
1-df_active_both[-1]/df_active_both[2]

0.4268774703557312

# Question 2

# Question 3

AB test

1. Randomly assign clients to two groups(low/high margin).
2. Check the monthly churn rate for two groups.
3. Compare and calculate the p-value


Potential risks

1. Different factors of companies <- randomize these
    
    
    - Region
    - Account size
    - How long they had been using the service

2. Company might lose alot of money from the test

3. Other factors not in the data

# Question 4

Compare proportion.

reject means/goodness of fit.
We are not calculating mean, so we did not use AB test on means
We are only working with two groups with two outcomes, so we don't need goodness of fit.

# Question 5

Null Hypothesis: Churn rate is the same bewteen two margin groups

# Question 6

Alternate Hypothesis: Churn rate is not the same bewteen two margin groups

# Question 7

In [197]:
df_active = df_nb.groupby('Margin Group').sum()

In [198]:
df_active

,Client number,Churned,active_prior_to_090916,active_prior_to_100116,active_prior_to_110116,active_prior_to_120116,active_prior_to_010117,active_prior_to_012717
Margin Group,,,,,,,,
High,736411,24.0,131.0,127.0,119.0,113.0,108.0,69.0
Low,690157,22.0,122.0,119.0,108.0,104.0,100.0,76.0


In [199]:
df_churn = pd.DataFrame([1-df_active.iloc[:,i+1]/df_active.iloc[:,i] for i in range(2,7)])

In [200]:
df_churn['n_high'] = df_active.iloc[0,2:-1].values
df_churn['n_low'] = df_active.iloc[1,2:-1].values

In [201]:
df_churn.iloc[0,2]

131.0

In [209]:
df_churn.loc[5] = [1-df_churn.iloc[0,-1]/df_churn.iloc[0,2], 1-df_churn.iloc[1,-1]/df_churn.iloc[1,2],
                   df_churn.iloc[0,2], df_churn.iloc[1,2]]

In [210]:
df_churn

Margin Group,High,Low,n_high,n_low
0,0.030534,0.024590,131.0,122.0
1,0.062992,0.092437,127.0,119.0
2,0.050420,0.037037,119.0,108.0
3,0.044248,0.038462,113.0,104.0
4,0.361111,0.240000,108.0,100.0
5,0.068702,0.062992,131.0,127.0


In [211]:
def test_stat(p1, p2, n1, n2):
    return (p1 - p2) / np.sqrt((p1*(1-p1)/n1) + (p2*(1-p2)/n2))

In [212]:
df_churn['test_stat'] = df_churn.apply(lambda line: test_stat(line['Low'], line['High'], line['n_low'], line['n_high']), axis=1)

In [213]:
df_churn['pval'] = df_churn['test_stat'].apply(lambda x: norm.cdf(x))

# Question 8

In [214]:
df_churn

Margin Group,High,Low,n_high,n_low,test_stat,pval
0,0.030534,0.024590,131.0,122.0,-0.289163,0.386228
1,0.062992,0.092437,127.0,119.0,0.860926,0.805360
2,0.050420,0.037037,119.0,108.0,-0.494462,0.310490
3,0.044248,0.038462,113.0,104.0,-0.214181,0.415203
4,0.361111,0.240000,108.0,100.0,-1.924533,0.027144
5,0.068702,0.062992,131.0,127.0,-0.184954,0.426633
